### my files

In [16]:
!dir

 Volume in drive C is OSDisk
 Volume Serial Number is 208B-1825

 Directory of C:\Users\edubov\Documents\git\public_github\information_retrieval_course\IR_chalenge

01/02/2020  10:59 AM    <DIR>          .
01/02/2020  10:59 AM    <DIR>          ..
01/02/2020  10:55 AM    <DIR>          .ipynb_checkpoints
01/02/2020  10:59 AM             3,975 ir_chalenge.ipynb
12/25/2019  08:39 PM           135,874 IR_Challenge.pdf
10/02/2019  10:59 AM         1,334,142 qrels_50_Queries
10/02/2019  10:59 AM            22,431 queriesROBUST.xml
12/29/2019  07:17 PM    <DIR>          ROBUSTindex
               4 File(s)      1,496,422 bytes
               4 Dir(s)  300,219,920,384 bytes free


### running indri query from python

In [17]:
!indrirunquery -index=ROBUSTindex -query=bobb

-5.2057	LA030490-0016	0	985
-5.21559	FT921-14992	0	1572
-5.33103	LA112290-0103	0	1037
-5.38052	FBIS3-8029	0	2085
-5.51988	LA042189-0102	0	1273
-5.52177	LA102690-0168	0	780
-5.53329	LA100590-0088	0	818
-5.57103	LA012989-0182	0	1471
-5.57169	LA123189-0085	0	685
-5.5752	LA071290-0107	0	960
-5.61577	LA082490-0135	0	1378
-5.7123	LA081489-0079	0	1771
-5.74349	LA052090-0241	0	1282
-5.7909	LA111290-0102	0	811
-5.79722	LA112290-0201	0	832
-5.82999	LA041290-0154	0	943
-5.84698	LA092390-0243	0	1002
-5.8541	LA061889-0101	0	2073
-5.86019	FT934-4622	0	347
-5.89002	LA100290-0099	0	1156
-5.89302	LA080689-0011	0	442
-5.8986	LA121690-0076	0	823
-5.91727	LA102190-0020	0	1257
-5.93886	LA011890-0039	0	580
-5.98352	LA061889-0086	0	2308
-5.98734	LA050590-0138	0	733
-5.98917	LA080890-0143	0	1138
-5.99902	LA072089-0185	0	771
-6.00796	FT934-16259	0	1207
-6.01461	LA082090-0121	0	413
-6.10372	LA040189-0007	0	237
-6.13926	FT921-14939	0	336
-6.15187	FT921-14793	0	372
-6.15475	LA040990-0146	0	3206
-6.15966	LA020390-

-7.31515	LA021689-0119	0	680
-7.31547	FT921-7962	0	681
-7.31578	LA060390-0017	0	682
-7.3161	LA042089-0143	0	683
-7.31641	LA022590-0009	0	684
-7.31735	LA121590-0053	0	687
-7.31766	LA052689-0026	0	688
-7.31861	LA080989-0032	0	691
-7.31923	LA111990-0132	0	693
-7.31923	LA062389-0014	0	693
-7.31934	LA121789-0008	0	5221
-7.32048	LA122090-0144	0	697
-7.32111	LA031590-0254	0	699
-7.32142	LA021790-0032	0	700
-7.32142	LA090989-0134	0	700
-7.32361	LA070689-0138	0	707
-7.32392	LA101889-0009	0	708
-7.32423	LA112590-0201	0	709
-7.32454	LA101990-0136	0	710
-7.32516	LA080190-0052	0	712
-7.32516	LA010589-0143	0	712
-7.32548	LA051390-0115	0	713
-7.32548	FT933-2299	0	713
-7.32641	LA082589-0137	0	716
-7.32672	LA061489-0028	0	717
-7.32734	FT941-9066	0	719
-7.32734	LA040990-0135	0	719
-7.3292	LA100790-0193	0	725
-7.33013	LA051990-0028	0	728
-7.33044	LA020289-0189	0	729
-7.33106	LA101189-0065	0	731
-7.33137	LA011090-0040	0	732
-7.33137	LA021690-0088	0	732
-7.33199	LA090590-0054	0	734
-7.33199	LA110190-0126	0

### indriquery output to pandas DataFrame

In [55]:
import pandas as pd

qres = !indrirunquery -index=ROBUSTindex -query=bobb
qres_df = pd.DataFrame.from_records([a.split('\t') for a in qres])
qres_df.columns = ['score', 'docid', 'ext_begin', 'ext_end']
qres_df.head()

# indrirunquery output format
# * score: the score of the returned document. An Indri query will always return a negative value for a result.
# * docID: the document ID
# * extent_begin: the starting token number of the extent that was retrieved
# * extent_end: the ending token number of the extent that was retrieved

,score,docid,ext_begin,ext_end
0,-5.74828,LA032890-0110,0,1264
1,-6.90624,LA051190-0116,0,496
2,-6.94036,LA091590-0120,0,600
3,-6.98527,LA011490-0143,0,1823
4,-7.20127,LA051190-0032,0,183


### training queries set 

In [56]:
import pandas as pd

a = pd.read_csv('qrels_50_Queries', sep=' ', header=None)
a.columns = ['qid', 'qother', 'docid', 'is_doc_rel']

print(a.head())
print(a.shape)
print(a['qid'].value_counts().sort_index())

   qid  qother        docid  is_doc_rel
0  301       0  FBIS3-10082           1
1  301       0  FBIS3-10169           0
2  301       0  FBIS3-10243           1
3  301       0  FBIS3-10319           0
4  301       0  FBIS3-10397           1
(61511, 4)
301    1512
302     874
303     761
304    1080
305    1463
306    1318
307    1236
308    1149
309    1057
310    1399
311    1299
312    1248
313     968
314    1438
315    1417
316    1521
317    1072
318    1393
319    1454
320     960
321    1369
322    1504
323    1291
324     822
325    1148
326     778
327    1332
328    1006
329    1224
330    1277
331    1381
332    1243
333     978
334    1204
335     950
336    1518
337    1044
338    1230
339     932
340    1409
341    1134
342    1612
343    1549
344    1250
345     867
346    1371
347    1079
348    1668
349    1437
350    1285
Name: qid, dtype: int64


### all queries

In [57]:
with open('queriesROBUST.xml', 'r') as queries_file:
    print(queries_file.read())

<parameters>
	<query>
		<number>301</number>
		<text>#combine( international organized crime )</text>
	</query>
	<query>
		<number>302</number>
		<text>#combine( poliomyelitis post polio )</text>
	</query>
	<query>
		<number>303</number>
		<text>#combine( hubble telescope achievements )</text>
	</query>
	<query>
		<number>304</number>
		<text>#combine( endangered species mammals )</text>
	</query>
	<query>
		<number>305</number>
		<text>#combine( dangerous vehicles )</text>
	</query>
	<query>
		<number>306</number>
		<text>#combine( african civilian deaths )</text>
	</query>
	<query>
		<number>307</number>
		<text>#combine( new hydroelectric projects )</text>
	</query>
	<query>
		<number>308</number>
		<text>#combine( implant dentistry )</text>
	</query>
	<query>
		<number>309</number>
		<text>#combine( rap crime )</text>
	</query>
	<query>
		<number>310</number>
		<text>#combine( radio waves brain cancer )</text>
	</query>
	<query>
		<number>311</number>
		<text>#combine( industrial e

In [ ]:
!